# Modelling for predicting the spring onset of natural vegetation across the Northern Hemisphere

https://doi.org/10.1016/j.ecolind.2021.108126


## Data

- Observation data from the USA National Phenology Network (USA-NPN)
- MODIS land cover type product and the land cover dynamics product
- Daymet gridded climate data covering Northern America
- Climate data obtained from Global Land Data Assimilation System (GLDAS) covering the Northern Hemisphere

### USA-NPN

- Observation records of vegetation phenology from 2895 sites from 2007 to 2017
- Four vegetation types defined in the USA-NPN dataset, including deciduous broadleaf forests (DBF), evergreen broadleaf forests (EBF), graminoid (GRA), and drought deciduous broadleaf forests (DDBF)
- Spring onset: breaking leaf buds for DBF, breaking leaf buds for ENF, breaking leaf buds for DDBF, and initial growth of grasses/sedges for GRA
- Averaging the timing of spring onsets for all records at the same site in the same year
- Excluding observation data in the sites located in the pixels that are classified as urban and build-up in the MODIS land cover maps

In [1]:
import springtime
from springtime.datasets.rnpn import RNPN, npn_species, npn_phenophases

# Find species

In [4]:
# List IDs and names for available species, phenophases
species = npn_species()
species.head()

,species_id,common_name,genus,genus_id,genus_common_name,species,kingdom,itis_taxonomic_sn,functional_type,class_id,class_common_name,class_name,order_id,order_common_name,order_name,family_id,family_name,family_common_name
1,120,'ohi'a lehua,Metrosideros,798,Lehuas (Metrosideros),polymorpha,Plantae,27259,Evergreen broadleaf,15,Flowering Plants,Magnoliopsida,89,Myrtle and Evening-primrose Families,Myrtales,301,Myrtaceae,Myrtle Family
2,1436,absinthium,Artemisia,437,Sagebrushes (Artemisia),absinthium,Plantae,35445,Forb,15,Flowering Plants,Magnoliopsida,69,"Aster, Bellflower and Buckbean Families",Asterales,242,Asteraceae,Aster Family
3,1227,Acadian flycatcher,Empidonax,612,Empidonax Flycatchers (Empidonax),virescens,Animalia,178339,Bird,5,Birds,Aves,31,Perching Birds,Passeriformes,154,Tyrannidae,Tyrant Flycatchers
4,1229,acorn woodpecker,Melanerpes,790,Melanerpine Woodpeckers (Melanerpes),formicivorus,Animalia,178189,Bird,5,Birds,Aves,33,Woodpeckers,Piciformes,158,Picidae,Woodpeckers
5,2110,Adam and Eve,Aplectrum,1285,Adam and Eves (Aplectrum),hyemale,Plantae,43489,Forb,15,Flowering Plants,Magnoliopsida,68,"Asparagas, Iris, Orchid and Aloe Families",Asparagales,307,Orchidaceae,Orchid Family


## Deciduous broadleaf

In [8]:
deciduous_broadleaf = species.query('functional_type=="Deciduous broadleaf"')
deciduous_broadleaf.head()

,species_id,common_name,genus,genus_id,genus_common_name,species,kingdom,itis_taxonomic_sn,functional_type,class_id,class_common_name,class_name,order_id,order_common_name,order_name,family_id,family_name,family_common_name
11,1174,Allegheny serviceberry,Amelanchier,402,Serviceberries (Amelanchier),laevis,Plantae,182046,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,96,"Oleaster, Buckthorn, Rose and Elm Families",Rosales,325,Rosaceae,Rose Family
24,1446,alternateleaf dogwood,Cornus,560,Dogwoods (Cornus),alternifolia,Plantae,27813,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,75,"Dogwood, Hydrangea and Tupelo Families",Cornales,262,Cornaceae,Dogwood Family
27,93,American basswood,Tilia,1056,Lindens (Tilia),americana,Plantae,21536,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,88,Mallow and Mezereum Families,Malvales,294,Malvaceae,Mallow Family
28,812,American beautyberry,Callicarpa,477,Beautyberries (Callicarpa),americana,Plantae,32144,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,83,"Mint, Olive and Plantain Families",Lamiales,286,Lamiaceae,Mint Family
29,79,American beech,Fagus,645,Beeches (Fagus),grandifolia,Plantae,19462,Deciduous broadleaf,15,Flowering Plants,Magnoliopsida,79,"Birch, Beech, Walnut and Bayberry Families",Fagales,271,Fagaceae,Beech Family


## Evergreen

In [22]:
evergreen_broadleaf = species.query('functional_type=="Evergreen broadleaf"')

# Phenophases

In [16]:
phenophases = npn_phenophases()
phenophases.head()

,phenophase_id,phenophase_name,phenophase_category,color,pheno_class_id
1,56,First leaf,Leaves,NA_character_,1
2,57,75% leaf elongation,Leaves,NA_character_,2
3,58,First flower,Flowers,NA_character_,7
4,59,Last flower,Flowers,NA_character_,9
5,60,First fruit ripe,Fruits,NA_character_,12


In [19]:
def lookup(df, column, expression):
    """Return rows where column matches expression."""
    return df[df[column].str.lower().str.contains(expression)]

In [20]:
breaking_leaf_buds = lookup(phenophases, "phenophase_name", "breaking leaf buds")
breaking_leaf_buds

,phenophase_id,phenophase_name,phenophase_category,color,pheno_class_id
100,371,Breaking leaf buds,Leaves,Green1,1
101,373,Breaking leaf buds,Leaves,Green1,1


In [21]:
initial_growth = lookup(phenophases, "phenophase_name", "initial growth")
initial_growth

,phenophase_id,phenophase_name,phenophase_category,color,pheno_class_id
138,482,Initial growth,Leaves,Green1,1
148,492,Initial growth,Leaves,Green1,1
164,508,Initial growth,Leaves,Green1,1


# Species and Phenophases summary

**Species id**: 
- DBF (1174,1446,...)
- EBF(120, 1861,...)
- GRA(1677,1032,1971,...)

**Phenophases**:

- breaking leaf buds (371,373)
- initial growth (482,492,508)

### Data preparation

In [14]:
# Load dataset
# years is needed! [start,end+1]!
# If there is no data for one year we get error---check years=[2013, 2014]

# A lot of queries returned no data Error! For example the one below

# dataset = NPNPhenor(species=1446, phenophase=373, years=[2007, 2018])
# dataset.download()
# df1 = dataset.load()
# df1

In [27]:
npn_broadleaf = RNPN(years=(2007, 2018), species_ids=deciduous_broadleaf.species_id.to_list(), phenophase_ids=breaking_leaf_buds.phenophase_id.to_list())
npn_broadleaf

RNPN(dataset='RNPN', years=(2007, 2018), species_ids=[1174, 1446, 93, 812, 79, 1832, 724, 1048, 71, 823, 91, 206, 1762, 970, 76, 1245, 1246, 810, 444, 1208, 1362, 1007, 1353, 20, 717, 1447, 2087, 1173, 1857, 320, 982, 72, 1485, 1878, 1875, 1880, 802, 1, 1188, 778, 87, 1176, 873, 28, 1849, 1481, 90, 839, 1458, 33, 1591, 1159, 80, 1494, 941, 1510, 1691, 1015, 2093, 2046, 328, 2, 799, 2042, 101, 1452, 2131, 2143, 765, 713, 1211, 1161, 1632, 64, 1373, 1181, 1649, 871, 1682, 1358, 1049, 2066, 2052, 1212, 1748, 1045, 2050, 29, 293, 1807, 89, 201, 829, 36, 1364, 303, 2101, 1431, 92, 897, 1499, 2145, 1759, 1489, 1732, 835, 944, 322, 1493, 430, 2053, 1000, 1817, 804, 976, 1733, 1044, 7, 1780, 1736, 1781, 1760, 1363, 1771, 836, 1006, 2142, 12, 73, 1066, 1367, 1843, 977, 988, 1871, 996, 1744, 1163, 1636, 62, 1805, 1448, 319, 75, 77, 2071, 1162, 1608, 94, 95, 885, 949, 1716, 1487, 942, 1250, 1438, 227, 1835, 1374, 1923, 1741, 1179, 1839, 1344, 216, 789, 214, 1169, 1775, 1503, 884, 723, 1214, 1509,

In [28]:
npn_broadleaf.download()

OSError: [Errno 36] File name too long: '/tmp/data/rnpn/rnpn_npn_data_y_2007_2018_s_1174_1446_93_812_79_1832_724_1048_71_823_91_206_1762_970_76_1245_1246_810_444_1208_1362_1007_1353_20_717_1447_2087_1173_1857_320_982_72_1485_1878_1875_1880_802_1_1188_778_87_1176_873_28_1849_1481_90_839_1458_33_1591_1159_80_1494_941_1510_1691_1015_2093_2046_328_2_799_2042_101_1452_2131_2143_765_713_1211_1161_1632_64_1373_1181_1649_871_1682_1358_1049_2066_2052_1212_1748_1045_2050_29_293_1807_89_201_829_36_1364_303_2101_1431_92_897_1499_2145_1759_1489_1732_835_944_322_1493_430_2053_1000_1817_804_976_1733_1044_7_1780_1736_1781_1760_1363_1771_836_1006_2142_12_73_1066_1367_1843_977_988_1871_996_1744_1163_1636_62_1805_1448_319_75_77_2071_1162_1608_94_95_885_949_1716_1487_942_1250_1438_227_1835_1374_1923_1741_1179_1839_1344_216_789_214_1169_1775_1503_884_723_1214_1509_1779_1754_2059_1177_1471_1758_2140_1184_1199_1033_1248_2097_983_1361_1371_1342_1009_1674_827_1370_1001_1761_102_915_779_1822_312_783_1840_872_31

In [19]:
# In totall there are 1755 species!
print(len(ls_sp1))
print(len(ls_sp2))

338
137


In [20]:
#ls_species = ls_sp1
ls_species = ls_sp2
ls_phases = [371,373]
with tempfile.TemporaryDirectory() as tmpdirname:
    for i in ls_species:
        for j in ls_phases:
            try:
                dataset = NPNPhenor(species=i, phenophase=j, years=[2007, 2018])
                dataset.download()
                df1 = dataset.load()
                df1.to_csv(tmpdirname + "/species_" + str(i)+str(j)+ ".csv")
                print(df1.shape)
            except:
                pass

    # list all CSV files in working directory.
    os.chdir(tmpdirname)
    all_filenames = [i for i in glob.glob('*.csv')]
    print(all_filenames)
    
    # combine all files in the list
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
    combined_csv.to_csv( current_path + "/combined_csv.csv", index=False, encoding='utf-8-sig')

downloading /tmp/data/NPN/phenor_npn_data_103_371_2007-01-01_2007-12-31.rds
  |======================================================================| 100%


R[write to console]: No encoding supplied: defaulting to UTF-8.

R[write to console]: Error in (function (species = 3, phenophase = 371, start = "2000-01-01",  : 
  Query returned no data, check your input parameters!



downloading /tmp/data/NPN/phenor_npn_data_103_373_2007-01-01_2007-12-31.rds
  |======================================================================| 100%


R[write to console]: No encoding supplied: defaulting to UTF-8.

R[write to console]: Error in (function (species = 3, phenophase = 371, start = "2000-01-01",  : 
  Query returned no data, check your input parameters!



downloading /tmp/data/NPN/phenor_npn_data_104_371_2007-01-01_2007-12-31.rds
  |======================================================================| 100%


...

R[write to console]: No encoding supplied: defaulting to UTF-8.

R[write to console]: Error in (function (species = 3, phenophase = 371, start = "2000-01-01",  : 
  Query returned no data, check your input parameters!



downloading /tmp/data/NPN/phenor_npn_data_2139_373_2007-01-01_2007-12-31.rds
  |======================================================================| 100%


R[write to console]: No encoding supplied: defaulting to UTF-8.

R[write to console]: Error in (function (species = 3, phenophase = 371, start = "2000-01-01",  : 
  Query returned no data, check your input parameters!



[]


ValueError: No objects to concatenate

In [ ]:
# dataset = NPNPhenor(species=36, phenophase=483, years=[2012, 2013])
# dataset.download()
# df1 = dataset.load()
# list(df1.columns)